In [1]:
import pandas as pd
from h3 import h3
from sqlalchemy import create_engine


In [ ]:
def h3_from_row(row,res,x,y):
    '''
    Esta funcion toma una fila, un nivel de resolucion de h3
    y los nombres que contienen las coordenadas xy
    y devuelve un id de indice h3
    '''
    return h3.geo_to_h3(row[y],row[x],res = res)

def h3_indexing(df,res_list,lat='LATITUDE',lon='LONGITUDE'):
    """
    Esta funcion toma una tabla con dos pares de coordenadas para origen y destino
    Un nivel de resolucion h3
    y devuelve la tabla con los ids de h3
    """
    
    if len(res_list) == 1:
        res_list.append(res_list[0])
        
    for res in range(res_list[0],res_list[1] + 1):
        df['h3_res_' + str(res)] = df.apply(h3_from_row, axis=1, args=[res,lon,lat])

    return df

In [ ]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [ ]:
%time trx = pd.read_csv('../data/original/D_USOS_DIA20181010.CSV',sep=';',parse_dates=[3])

In [ ]:
# limpiar latlong nulls 
%time trx = trx.dropna(subset=['LATITUDE','LONGITUDE'])

In [ ]:
# limpiar fuera del area de interes
trx = trx.loc[(trx.LATITUDE > -35.55) & (trx.LATITUDE < -33.9) & (trx.LONGITUDE > -60.1 ) & (trx.LONGITUDE < -57.1),:]

In [ ]:
trx.head()

In [ ]:
trx.shape

In [ ]:
%time trx_h3 = h3_indexing(trx.copy(),res_list = [10,11])

In [ ]:
trx_h3.head()

In [ ]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'
    .format(DB_USERNAME, DB_PASSWORD, DB_HOST,
            DB_PORT, DB_NAME))

In [ ]:
trx_h3.shape

In [ ]:
3649378 + 3661996 + 3660895

In [ ]:
%time trx_h3.to_sql('trx', engine, schema=DB_SCHEMA,if_exists='append',index=False)